In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd
from IPython.display import HTML

conexao = mysql.connector.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    password='12345678',
    database='usando_sql_em_python'
)

# Função para lidar com a seleção do usuário
def handle_selection(selection, esn, ignition_filter, cursor):
    consulta = "SELECT * FROM tabela_dados WHERE esn = %s"
    if selection == options[0]:
        cursor.execute(consulta, (esn,))
        return cursor.fetchall()  
    elif selection == options[1]:
        consulta += " AND battery < 10000"
    elif selection == options[2]:
        consulta += " AND ignition = 1"
    elif selection == options[3]:
        consulta += " AND speed > 10"

    if ignition_toggle.value != 'Sem filtro':
        ignition_filter = f" AND ignition = {'1' if ignition_toggle.value == 'Ligado' else '0'}"
    else:
        ignition_filter = ''

    if speed_toggle.value != 'Sem filtro':
        speed_filter = f" AND speed {'<' if speed_toggle.value == 'Parado' else '>'} 5"
    else:
        speed_filter = ''
        
    consulta += ignition_filter
    consulta += speed_filter


        
    cursor.execute(consulta, (esn,))
    return cursor.fetchall()


def handle_dropdown_change(change):
    new_value = change['new']
    if new_value == options[0]:
        clear_output()
        display(esn)
        display(dropdown)
        display(button)
    elif new_value == options[1]:
        clear_output()
        display(esn)
        display(dropdown)
        display(ignition_toggle)
        display(speed_toggle)
        display(button)
    else:
        clear_output()
        display(esn)
        display(dropdown)
        display(button)


# Função de callback para o evento de clique do botão
def on_button_click(button):
    clear_output()
    cursor = conexao.cursor()
    resultado = handle_selection(dropdown.value, esn.value, ignition_toggle.value, cursor)
    if resultado:
        df = pd.DataFrame(resultado, columns=[desc[0] for desc in cursor.description])
        df_head = df.head(50)
        styled_df = df_head.style.set_table_styles([{'selector': 'tr th:first-child', 'props': [('display', 'none')]}])
        export_button.result = df  # Passando o DataFrame para o atributo result do botão de exportação

        handle_dropdown_change({'new': dropdown.value})
        display(export_button)
        display(styled_df)
        

# Função de callback para o evento de clique do botão de exportação
def export_table(button):
    df = button.result  # Obtendo o DataFrame do atributo result do botão de exportação
    df.to_csv('tabela.csv', index=False)
    print("Tabela exportada como 'tabela.csv'")

try:
    if conexao.is_connected():

        # Campo de busca ESN
        esn = widgets.Text(
            placeholder='Informe o ESN',
            description='ESN:',
            disabled=False   
        )

        # Botão de busca
        button = widgets.Button(
            description='Buscar',
            disabled=False,
            button_style='info',
            icon='check'
        )

        # Botão de exportação
        export_button = widgets.Button(
            description='Exportar Tabela',
            disabled=False,
            button_style='success',
            icon='download'
        )

        ignition_toggle = widgets.ToggleButtons(
            options=['Sem filtro', 'Ligado', 'Desligado'],
            value='Sem filtro',
            description='IGNIÇÃO',
            disabled=False,
        )

        speed_toggle = widgets.ToggleButtons(
            options=['Sem filtro', 'Em movimento', 'Parado'],
            value='Sem filtro',
            description='VELOCIDADE',
            disabled=False
        )

        export_button.on_click(export_table)
        export_button.result = None  # Inicializando o atributo result do botão de exportação como None

        # Criar o menu de opções
        options = ["Sem filtro", "Desconexão da bateria", "teste", "Velocidade maior que 10"]
        dropdown = widgets.Dropdown(options=options, description="Escolha o filtro desejado")

        dropdown.observe(handle_dropdown_change, names='value')




        # Registrar a função de callback para o evento de clique do botão
        button.on_click(on_button_click)

        # Exibir os widgets
        display(esn)
        display(dropdown)
        display(button)

except mysql.connector.Error as erro:
    # Se ocorrer um erro durante a conexão, imprime a mensagem de erro
    print(f"Erro ao conectar ao banco de dados: {erro}")


Text(value='1', description='ESN:', placeholder='Informe o ESN')

Dropdown(description='Escolha o filtro desejado', index=1, options=('Sem filtro', 'Desconexão da bateria', 'te…

ToggleButtons(description='IGNIÇÃO', index=1, options=('Sem filtro', 'Ligado', 'Desligado'), value='Ligado')

ToggleButtons(description='VELOCIDADE', index=2, options=('Sem filtro', 'Em movimento', 'Parado'), value='Para…

Button(button_style='info', description='Buscar', icon='check', style=ButtonStyle())

Button(button_style='success', description='Exportar Tabela', icon='download', style=ButtonStyle())

,esn,timestamp,battery,ignition,speed,imobilizer
0,1,2024-01-18 12:58:11,6040.15,1,0,disabled
